# DeepSeek OCR - Experimento com modelo quantizado (4-bit)


---
---


Este notebook demonstra o uso do modelo **DeepSeek-OCR** quantizado em 4-bit para extração de texto de imagens

## Sobre o experimento
- **Modelo**: DeepSeek-OCR quantizado em 4-bit (NF4)
- **Objetivo**: Realizar OCR (Optical Character Recognition) em imagens e documentos
- **Ambiente**: Google Colab com GPU T4

## Configuração inicial
Primeiro, vamos instalar todas as dependências necessárias usando o `uv`


In [ ]:
# Install uv package manager
!pip install -q uv


### Instalação das dependências principais

Agora vamos instalar todas as bibliotecas necessárias:
- **PyTorch**: Framework de deep learning
- **Transformers**: Biblioteca da Hugging Face para modelos de linguagem
- **BitsAndBytes**: Para quantização de modelos
- **Accelerate**: Para otimização de inferência
- **Pillow**: Para processamento de imagens
- Outras dependências auxiliares


In [ ]:
# Install core dependencies with specific versions
!uv pip install torch==2.6.0 transformers==4.46.3 tokenizers==0.20.3 einops addict easydict pillow torchvision bitsandbytes accelerate numpy


### Instalação do Flash Attention (Opcional)

**Importante**: O `flash-attn` requer GPUs modernas com suporte a CUDA compute capability >= 8.0

**GPUs compatíveis:**
- NVIDIA A100, A10, A6000
- RTX 30xx series (3090, 3080, etc.)
- RTX 40xx series

**GPUs NÃO compatíveis:**
- Tesla T4 (Google Colab) - compute capability 7.5
- GPUs mais antigas

Como estamos usando o ambiente do Google Colab com GPU T4, **não poderemos usar** `flash_attention_2`, para isso, usaremos a implementação `eager` no lugar


In [ ]:
# Uncomment the line below if you have a compatible GPU (compute capability >= 8.0)
# !uv pip install flash-attn==2.7.3 --no-build-isolation

---


## Importação das bibliotecas

Agora vamos importar todas as bibliotecas necessárias para o experimento


In [ ]:
from transformers import AutoModel, AutoTokenizer
from PIL import Image

import torch
import time
import os


---


## Carregando o modelo

Vamos carregar o modelo **DeepSeek-OCR** quantizado em 4-bit

A quantização reduz significativamente o uso de memória, permitindo rodar o modelo em GPUs com menos VRAM e ainda sim obter bons resultados


In [ ]:
# Set CUDA device
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Model configuration
model_name = 'Jalea96/DeepSeek-OCR-bnb-4bit-NF4'    # 4-bit quantized model

# If you want to use the non-quantized model, uncomment the line below and comment the line above
# model_name = 'deepseek-ai/DeepSeek-OCR'            # Non-quantized model

print('=' * 70)
print('CARREGANDO MODELO DEEPSEEK-OCR')
print(f'\nModelo: {model_name}')
print('=' * 70)

print('\nCarregando tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code = True
)
print('Tokenizer carregado com sucesso!')

print('\nCarregando modelo...')
model = AutoModel.from_pretrained(
    model_name,
    _attn_implementation = 'eager',   # Using 'eager' because T4 GPU doesn't support flash_attention_2
    trust_remote_code = True,
    use_safetensors = True,
    device_map = 'auto',
    torch_dtype = torch.bfloat16,
)
model = model.eval()
print('Modelo carregado com sucesso!')

print('\n' + '=' * 70)
print(f'Dispositivo: {next(model.parameters()).device}')
print(f'Tipo de dados: {next(model.parameters()).dtype}')
print('=' * 70 + '\n')


---


## Configuração dos parâmetros

Vamos configurar os parâmetros para o processamento de OCR:

### Modos de configuração disponíveis:

| Modo          | base_size | image_size    | crop_mode | Uso Recomendado                               |
|---------------|-----------|---------------|-----------|-----------------------------------------------|
| **Tiny**      | 512       | 512           | False     | Testes rápidos, documentos simples            |
| **Small**     | 640       | 640           | False     | Documentos de tamanho médio                   |
| **Base**      | 1024      | 1024          | False     | Documentos padrão                             |
| **Large**     | 1280      | 1280          | False     | Documentos complexos, alta qualidade          |
| **Gundam**    | 1024      | 640           | True      | **Recomendado para a maioria dos documentos** |

Neste experimento, vamos usar o modo **Large** para obter a melhor qualidade possível


In [ ]:
# OCR configuration
prompt = '<image>\nFree OCR. '

# File path
base_path = ''
image_file = os.path.join(base_path, 'data_experiments', 'case_01_drivers_license.jpeg')
output_path = os.path.join(base_path, 'output')
os.makedirs(output_path, exist_ok=True)

# Model inference configuration
base_size = 1280
image_size = 1280
crop_mode = False

print('CONFIGURAÇÃO DO EXPERIMENTO')
print('=' * 70)
print(f'Arquivo de entrada: {image_file}')
print(f'Diretório de saída: {output_path}')
print(f'\nParâmetros do modelo:')
print(f'   • Modo: Large')
print(f'   • Base size: {base_size}px')
print(f'   • Image size: {image_size}px')
print(f'   • Crop mode: {crop_mode}')
print('=' * 70 + '\n')


---


## Processamento de OCR

Agora vamos processar a imagem e extrair o texto usando o modelo DeepSeek-OCR


In [ ]:
print('\n' + '=' * 70)
print('INICIANDO PROCESSAMENTO DE OCR')
print('=' * 70 + '\n')

# Check if image file exists
if not os.path.exists(image_file):
    print(f'Erro: arquivo de imagem não encontrado: {image_file}')
else:
    print(f'Processando imagem: {os.path.basename(image_file)}')
    print('-' * 70)

    try:
        # Load and save the image
        temp_path = os.path.join(output_path, f'temp_case_01.png')
        Image.open(image_file).convert('RGB').save(temp_path)

        print('\nExecutando inferência do modelo...')
        start_time = time.time()

        # Run OCR inference
        result = model.infer(
            tokenizer,
            prompt = prompt,
            image_file = temp_path,
            output_path = output_path,
            base_size = base_size,
            image_size = image_size,
            crop_mode = crop_mode,
            save_results = True,
            test_compress = False
        )

        end_time = time.time()
        inference_time = end_time - start_time

        print(f'Inferência concluída!')
        print(f'Tempo de processamento: {inference_time:.2f} segundos')
        print(f'Resultados salvos em: {output_path}')

    except Exception as e:
        print(f'\nErro ao processar {os.path.basename(image_file)}: {str(e)}')
        import traceback
        traceback.print_exc()

print('\n' + '=' * 70)
print('PROCESSAMENTO CONCLUÍDO!')
print('=' * 70)


---


## Informações adicionais

### Sobre o modelo quantizado

O modelo utilizado (`Jalea96/DeepSeek-OCR-bnb-4bit-NF4`) é uma versão quantizada em 4-bit do DeepSeek-OCR original (`deepseek-ai/DeepSeek-OCR`). A quantização:

- **Reduz o uso de memória** em aproximadamente 75%
- **Permite execução em GPUs com menos VRAM** (como a T4 do Colab)
- **Acelera a inferência** em alguns casos
- **Pode ter pequena perda de precisão** em comparação com o modelo completo

### Implementação de camada de atenção

**Flash Attention 2** vs **Eager**:

- `flash_attention_2`: Otimizado para GPUs modernas (compute capability >= 8.0)
  - Mais rápido e eficiente em memória
  - Não suportado pela GPU T4 do Google Colab
  
- `eager`: Implementação padrão do PyTorch
  - Compatível com todas as GPUs
  - Mais lento que flash attention em GPUs compatíveis

### Arquivos de saída

Os seguintes arquivos são gerados no diretório `output/`:
- Arquivo de texto com o OCR extraído
- Arquivo temporário da imagem processada
- Possíveis visualizações intermediárias (dependendo da configuração)
